In [48]:
import os
import shutil
import pathlib
import regex as re
import pandas as pd

In [2]:
rawdata_dir = pathlib.Path('../data/raw/DOC_AucklandIsland/Auckland Island summer trial 2019')

In [3]:
cat_type = ['cat', 'collared']
interaction_type = ['eat', 'ignore', 'interact']

In [4]:
cat_prefix = pd.Series()
cat_img_names = pd.Series()

/var/folders/r7/hlvr4tcn43d2pztz8gqr8_sc0000gn/T/ipykernel_5454/3950471939.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cat_prefix = pd.Series()
/var/folders/r7/hlvr4tcn43d2pztz8gqr8_sc0000gn/T/ipykernel_5454/3950471939.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cat_img_names = pd.Series()


In [5]:
for catdir in list(rawdata_dir.iterdir()):
    if catdir.is_dir():
        for cat_interaction in list(catdir.iterdir()):
            if cat_interaction.is_dir():
                indiv_prefix = f"{catdir.name}-{cat_interaction.name}"
                filenames = [f.name for f in cat_interaction.iterdir() if f.is_file()]
                cat_img_names = cat_img_names.append(pd.Series(filenames))
                cat_prefix = cat_prefix.append(pd.Series([indiv_prefix] * len(filenames)))

/var/folders/r7/hlvr4tcn43d2pztz8gqr8_sc0000gn/T/ipykernel_5454/2886350447.py:7: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cat_img_names = cat_img_names.append(pd.Series(filenames))
/var/folders/r7/hlvr4tcn43d2pztz8gqr8_sc0000gn/T/ipykernel_5454/2886350447.py:8: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cat_prefix = cat_prefix.append(pd.Series([indiv_prefix] * len(filenames)))
/var/folders/r7/hlvr4tcn43d2pztz8gqr8_sc0000gn/T/ipykernel_5454/2886350447.py:7: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cat_img_names = cat_img_names.append(pd.Series(filenames))
/var/folders/r7/hlvr4tcn43d2pztz8gqr8_sc0000gn/T/ipykernel_5454/2886350447.py:8: FutureWarning: The series.append method is deprecated and will be removed f

In [14]:
cat_img_timestamps = cat_img_names.str.extract(r'(\d{14})')

True

In [19]:
cat_img_timestamps_1d = cat_img_timestamps.values.ravel()

In [40]:
cat_info_df = pd.DataFrame({'prefix': cat_prefix, 'img_name': cat_img_names, 'timestamp': cat_img_timestamps_1d})

In [41]:
cat_info_df.dropna(inplace=True)
cat_info_df.sort_values(by=['prefix', 'timestamp'], inplace=True)
cat_info_df.reset_index(drop=True, inplace=True)

In [42]:
duplicated_timestamps = cat_info_df[cat_info_df['timestamp'].duplicated(keep=False)]
duplicated_timestamps

,prefix,img_name,timestamp
20,cat-eat,20190801_4v26_AucklandIsland_A1_2_3_SD98_20190...,20190210155439
21,cat-eat,20190801_4v26_AucklandIsland_A1_2_3_SD98_20190...,20190210155439
26,cat-eat,20190801_4v26_AucklandIsland_A1_2_3_SD98_20190...,20190210155515
27,cat-eat,20190801_4v26_AucklandIsland_A1_2_3_SD98_20190...,20190210155515
40,cat-eat,20190801_4v26_AucklandIsland_A1_2_3_SD98_20190...,20190210155648
...,...,...,...
4162,collared-interact,20190801_kW8J_AucklandIsland_A2_3_41_SD15_2019...,20190220101232
4163,collared-interact,20190801_kW8J_AucklandIsland_A2_3_41_SD15_2019...,20190220101232
4165,collared-interact,20190801_kW8J_AucklandIsland_A2_3_41_SD15_2019...,20190220101236
4166,collared-interact,20190801_kW8J_AucklandIsland_A2_3_41_SD15_2019...,20190220101236


In [44]:
grouped_indices = cat_info_df.groupby(cat_info_df['timestamp'].str[:8]).apply(lambda x: x.index.tolist())
for i in grouped_indices.index:
    print(i, len(grouped_indices[i]))

20190130 272
20190202 249
20190204 46
20190206 33
20190208 29
20190209 18
20190210 340
20190211 454
20190212 99
20190213 60
20190214 112
20190215 223
20190216 554
20190217 193
20190218 394
20190219 146
20190220 358
20190221 21
20190222 48
20190223 366
20190224 93
20190225 91
20190226 1
20190227 7
20190301 16
20190304 4


In [45]:
regrouped_dir_path = pathlib.Path("../data/raw") / "regrouped"
regrouped_dir_path.mkdir(parents=True, exist_ok=True)

In [51]:
for date_id in grouped_indices.index:
    date_dir_path = regrouped_dir_path / date_id
    date_dir_path.mkdir(parents=True, exist_ok=True)

    for idx in grouped_indices[date_id]:
        folder_name = cat_info_df.loc[idx, 'prefix'].split('-')[0]
        interaction_type = cat_info_df.loc[idx, 'prefix'].split('-')[1]
        img_path = rawdata_dir / folder_name/ interaction_type / cat_info_df.loc[idx, 'img_name']
        shutil.copy(img_path, date_dir_path)

## Labelling

In [52]:
cat_info_fk = cat_info_df.copy()
cat_info_fk['regrp_folder'] = pd.Series()

timestamp
20190130    [2757, 2758, 2759, 2760, 2761, 2762, 2763, 276...
20190202    [420, 421, 422, 2887, 2888, 2889, 2890, 2891, ...
20190204    [423, 424, 425, 426, 427, 428, 429, 430, 431, ...
20190206    [469, 470, 471, 472, 473, 474, 475, 476, 477, ...
20190208    [502, 503, 504, 505, 506, 507, 508, 509, 510, ...
20190209    [531, 532, 533, 534, 535, 536, 537, 538, 539, ...
20190210    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
20190211    [263, 264, 265, 266, 267, 268, 269, 270, 271, ...
20190212    [630, 631, 632, 633, 634, 635, 636, 637, 638, ...
20190213    [703, 704, 705, 706, 707, 708, 709, 710, 711, ...
20190214    [741, 742, 743, 744, 745, 746, 747, 748, 749, ...
20190215    [799, 800, 801, 802, 803, 804, 805, 806, 807, ...
20190216    [1000, 1001, 1002, 1003, 1004, 1005, 1006, 100...
20190217    [1551, 1552, 1553, 1554, 1555, 1556, 1557, 155...
20190218    [1599, 1600, 1601, 1602, 1603, 1604, 1605, 160...
20190219    [1630, 1631, 1632, 1633, 1634, 1635, 1636, 163..

In [83]:
indices_regrouped = pd.Series()
folder_name = pd.Series()

/var/folders/r7/hlvr4tcn43d2pztz8gqr8_sc0000gn/T/ipykernel_5454/4101975384.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  indices_regrouped = pd.Series()
/var/folders/r7/hlvr4tcn43d2pztz8gqr8_sc0000gn/T/ipykernel_5454/4101975384.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  folder_name = pd.Series()


In [84]:
for idx in grouped_indices.index:
    indices_regrouped = pd.concat([indices_regrouped, pd.Series(grouped_indices[idx])])
    folder_name = pd.concat([folder_name, pd.Series([idx]*len(grouped_indices[idx]))])
        

In [85]:
folder_group = pd.DataFrame({'index': indices_regrouped, 'folder_name': folder_name}).set_index('index')

In [99]:
expanded_cat_info = pd.merge(cat_info_df, folder_group, left_index=True, right_index=True).sort_values(by=['folder_name', 'timestamp'])

In [110]:
labelled_df = pd.read_csv('../data/labels/init_labels.csv', na_values='void').dropna()

In [111]:
labelled_df['label'].value_counts()

linja         1412
matt          1395
bethilda       883
nuclei         345
taki            96
neapolitan      42
Name: label, dtype: int64

In [112]:
labelled_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4173 entries, 0 to 4226
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   4173 non-null   int64  
 1   prefix       4173 non-null   object 
 2   img_name     4173 non-null   object 
 3   timestamp    4173 non-null   float64
 4   folder_name  4173 non-null   int64  
 5   label        4173 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 228.2+ KB
